In [ ]:
## Preparation

# Import the necessary libraries
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sympy.strategies.core import switch
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader, Subset
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report, roc_curve, auc
from torchvision.models import efficientnet_b0
from tqdm import tqdm

import matplotlib.pyplot as plt
import numpy as np
import os
import optuna
import time

dataset_path = os.getcwd() + '/dataset/classified'

class_labels = ["Normal", "Cataract"]

In [ ]:
# 1. Load checkpoint
model_path = r"H:\Kerja\Project\cataract detection\cataract detection - dr arya\cataract-detection\output\checkpoint-94.82758620689656-hyperparam.pth"
checkpoint = torch.load(model_path, map_location=torch.device('cuda'))

# 2. Recreate model
model = efficientnet_b0(weights=None)  # Jangan pakai pretrained (karena akan overwrite)
model.classifier = nn.Sequential(
    nn.Linear(number_of_features, 256),
    nn.BatchNorm1d(256),
    Swish(),

    ResidualBlock(256, 256),
    nn.Dropout(dropout_rate),

    ResidualBlock(256, 128),
    nn.Dropout(dropout_rate),

    ResidualBlock(128, 128),
    nn.Dropout(dropout_rate),

    ResidualBlock(128, 64),
    nn.Dropout(dropout_rate),

    nn.Linear(64, 1)
)

num_classes = 2  # Ganti sesuai jumlah kelas kamu
model.classifier[1] = torch.nn.Linear(model.classifier[1].in_features, num_classes)

# 3. Load weights dari checkpoint
model.load_state_dict(checkpoint['model_state_dict'])
model = model.to(torch.device('cuda'))  # Optional, kalau pakai GPU
model.eval()

# 4. Tes
print(model)

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # typical ImageNet means
                         std=[0.229, 0.224, 0.225])   # typical ImageNet stds
])

In [ ]:
def classify_image(image_path):
    image = Image.open(image_path).convert('RGB')
    input_tensor = transform(image).unsqueeze(0)

    print(input_tensor)

    with torch.no_grad():
        output = model(input_tensor)
        prediction = torch.argmax(output, dim=1).item()

    return prediction

In [ ]:
image_path = './dataset/additional_penelitian/Katarak/OD 1.jpg'
prediction = classify_image(image_path)
print(f"Predicted class {prediction}")